In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
from prophet import Prophet
import plotly.graph_objects as go
import plotly.io as pio

In [3]:
main_frame = pd.read_csv('/content/sales.csv')

In [4]:
main_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320428 entries, 0 to 320427
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   product_id                  320428 non-null  object 
 1   product_name                319763 non-null  object 
 2   category_id                 319763 non-null  object 
 3   category_name               319763 non-null  object 
 4   product_price               320428 non-null  float64
 5   product_stock               319763 non-null  float64
 6   quantity                    320428 non-null  float64
 7   order_id                    320428 non-null  object 
 8   order_date                  320428 non-null  object 
 9   order_total_products_price  320428 non-null  float64
 10  order_delivery_cost         6546 non-null    float64
 11  order_total_price           320428 non-null  float64
 12  order_customer_type         320428 non-null  object 
dtypes: float64(6),

In [5]:
main_frame.head()

,product_id,product_name,category_id,category_name,product_price,product_stock,quantity,order_id,order_date,order_total_products_price,order_delivery_cost,order_total_price,order_customer_type
0,65c69ee781e808fb28054a36,طماطم فاخر,633eb7d9685e55eb2f02b012,خضروات,9.0,1.000000e+14,4.0,65c76f779634263f010b3a62,2024-02-10 03:28:06,106.0,NaN,106.0,passing
1,65c6ac86b138a0346100f765,كرنب مسلوق,63750f318d0151836508a2c2,خضروات جاهزة,30.0,1.000000e+11,1.0,65c76f779634263f010b3a62,2024-02-10 03:28:06,106.0,NaN,106.0,passing
2,65c69f50c8da21dfe90ab455,جريب فروت,633eb7e11ef9e97ab70d6ff2,فاكهة,7.5,1.000000e+12,4.0,65c76f779634263f010b3a62,2024-02-10 03:28:06,106.0,NaN,106.0,passing
3,65c6ad06b41824de3a0e8105,ليمون اضاليا,633eb7d9685e55eb2f02b012,خضروات,10.0,1.000000e+12,1.0,65c76f779634263f010b3a62,2024-02-10 03:28:06,106.0,NaN,106.0,passing
4,65c69ee781e808fb28054a36,طماطم فاخر,633eb7d9685e55eb2f02b012,خضروات,9.0,1.000000e+14,4.0,65c76f77c5c149c84402d922,2024-02-10 03:28:06,106.0,NaN,106.0,passing


In [6]:
# Group the main_frame by order_id and sum the 'quantity'
aggregated_data = main_frame.groupby('order_id').agg({
    'quantity': 'sum',
    'order_total_products_price': 'first',  # Taking the first value of these columns
    'order_delivery_cost': 'first',
    'order_total_price': 'first',
    'order_customer_type': 'first',
    'order_date': 'first'  # Ensure consistency of the date for each unique order_id
}).reset_index()

# Create the new DataFrame (df) with the aggregated data
df = aggregated_data[['order_id', 'quantity', 'order_total_products_price', 'order_date']]

In [7]:
# Step 1: Convert 'order_date' column to datetime format
df['order_date'] = pd.to_datetime(df['order_date'])

df.loc[:, 'order_date'] = df['order_date'].dt.date

# Optionally, convert back to datetime (without time) if needed
df.loc[:, 'order_date'] = pd.to_datetime(df['order_date'])

<ipython-input-7-f533c2134a60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['order_date'] = pd.to_datetime(df['order_date'])


In [8]:
# Drop rows where 'order_id' is '6680f642b2509ea4dc0964e2'
df = df[df['order_id'] != '6680f642b2509ea4dc0964e2']

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90084 entries, 0 to 90084
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    90084 non-null  object        
 1   quantity                    90084 non-null  float64       
 2   order_total_products_price  90084 non-null  float64       
 3   order_date                  90084 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 3.4+ MB


In [10]:
# Define the date ranges you want to exclude
exclude_ranges = [
    ('2024-04-09', '2024-04-13'),
    ('2024-06-15', '2024-06-21'),
    ('2024-08-15', '2024-08-15'),  # Single day range
    ('2024-08-27', '2024-08-30')
]

# Create a mask to filter out rows within these date ranges
mask = pd.Series([True] * len(df), index=df.index)  # Initially, include all rows

for start_date, end_date in exclude_ranges:
    mask &= ~((df['order_date'] >= start_date) & (df['order_date'] <= end_date))

# Apply the mask to filter out the rows within the specified date ranges
df_filtered = df[mask]

# Check the result
df_filtered.shape
df = df_filtered
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89973 entries, 0 to 90084
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    89973 non-null  object        
 1   quantity                    89973 non-null  float64       
 2   order_total_products_price  89973 non-null  float64       
 3   order_date                  89973 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 3.4+ MB


In [11]:
df['order_date'].value_counts()

,count
order_date,
2024-08-12,721
2024-08-02,705
2024-08-11,658
2024-08-10,644
2024-07-11,636
...,...
2024-04-08,151
2024-04-14,148
2024-08-31,142


In [12]:
# Set the order_date as the index
df.set_index('order_date', inplace=True)

# Aggregate daily sales data (sum of quantity and total price)
daily_sales = df.resample('D').agg({
    'quantity': 'sum',
    'order_total_products_price': 'sum'
})

In [13]:
monthly_sales = df.resample('M').agg({
    'quantity': 'sum',
    'order_total_products_price': 'sum'
})

In [14]:
#Plot daily sales revenue with 7-day moving average of revenue

daily_sales['7_day_avg_revenue'] = daily_sales['order_total_products_price'].rolling(window=7).mean()

# Set default renderer to browser for better interactive display
pio.renderers.default = 'colab'

# Create initial figure

fig = go.Figure()

# Add traces for daily sales revenue
fig.add_trace(go.Scatter(
    x=daily_sales.index,
    y=daily_sales['order_total_products_price'],
    mode='lines',
    name='Daily Sales Revenue',
    line=dict(color='cyan')
))

# Add traces for 7-day moving average of revenue
fig.add_trace(go.Scatter(
    x=daily_sales.index,
    y=daily_sales['7_day_avg_revenue'],
    mode='lines',
    name='7-Day Moving Avg (Revenue)',
    line=dict(color='black')
))

# Customize the layout
fig.update_layout(
    title='Daily Sales Trend with 7-Day Moving Average',
    xaxis_title='Date',
    yaxis_title='Sales Revenue',
    hovermode='x',
    template='plotly_white',
    width=1000,  # Adjust figure size
    height=600
)

# Show the interactive plot
fig.show()


In [15]:
# Calculate the 7-day moving average for quantity sold
daily_sales['7_day_avg_quantity'] = daily_sales['quantity'].rolling(window=7).mean()

# Create the interactive plot for daily quantity sold
fig = go.Figure()

# Add a line plot for daily quantity sold
fig.add_trace(go.Scatter(
    x=daily_sales.index,
    y=daily_sales['quantity'],
    mode='lines',
    name='Daily Quantity Sold',
    line=dict(color='black')
))

# Add a line plot for the 7-day moving average of quantity sold
fig.add_trace(go.Scatter(
    x=daily_sales.index,
    y=daily_sales['7_day_avg_quantity'],
    mode='lines',
    name='7-Day Moving Avg (Quantity)',
    line=dict(color='red', dash='dash')  # Dashed line for the trend
))

# Customize the layout
fig.update_layout(
    title='Daily Sales Volume Trend With 7-Day Moving Average',
    xaxis_title='Date',
    yaxis_title='Quantity Sold',
    hovermode='x',
    template='plotly_white',
    width=1000,  # Adjust figure size
    height=600
)

# Show the interactive plot
fig.show()

In [16]:
# Calculate the 3-month rolling average for the quarterly trend
monthly_sales['3_month_avg_revenue'] = monthly_sales['order_total_products_price'].rolling(window=3).mean()

# Create the interactive plot
fig = go.Figure()

# Add a line plot for monthly sales revenue
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales['order_total_products_price'],
    mode='lines',
    name='Monthly Sales Revenue',
    line=dict(color='green')
))

# Add a line plot for the 3-month trend
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales['3_month_avg_revenue'],
    mode='lines',
    name='3-Month Moving AVG (Revenue)',
    line=dict(color='blue', dash='dash')  # Dashed line for the trend
))
# Customize the layout
fig.update_layout(
    title='Monthly Sales Trend with 3-Months Moving Average',
    xaxis_title='Date',
    yaxis_title='Quantity SOld',
    hovermode='x',
    template='plotly_white',
    width=1000,  # You can adjust the figure size
    height=600
)

# Show the interactive plot
fig.show()

In [17]:
# Plot Monthly quantity sold

# Calculate the 3-month rolling average for the quarterly trend
monthly_sales['3_month_avg_quantity'] = monthly_sales['quantity'].rolling(window=3).mean()

# Create the interactive plot
fig = go.Figure()

# Add a line plot for daily sales revenue
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales['quantity'],
    mode='lines',
    name='Monthly Quantity Sold',
    line=dict(color='purple')
))

# Add a line plot for the 3-month trend
fig.add_trace(go.Scatter(
    x=monthly_sales.index,
    y=monthly_sales['3_month_avg_quantity'],
    mode='lines',
    name='3-Month Moving AVG (Quantity)',
    line=dict(color='blue', dash='dash')  # Dashed line for the trend
))

# Customize the layout
fig.update_layout(
    title='Monthly Sales Volume Trend With 3-Months Moving Average',
    xaxis_title='Date',
    yaxis_title='Quantity SOld',
    hovermode='x',
    template='plotly_white',
    width=1000,  # You can adjust the figure size
    height=600
)

# Show the interactive plot
fig.show()

In [18]:
# Ensure 'order_total_products_price' is not empty and drop NaN values
sales_data = daily_sales['order_total_products_price'].dropna()

# Decompose the time series
decomposition = sm.tsa.seasonal_decompose(sales_data, model='additive')

# Extract the components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
dates = sales_data.index  # Use the index of the original series

# Create the interactive plot
fig = go.Figure()

# Add the trend component
fig.add_trace(go.Scatter(
    x=dates,
    y=trend,
    mode='lines',
    name='Trend',
    line=dict(color='blue')
))

# Add the seasonal component
fig.add_trace(go.Scatter(
    x=dates,
    y=seasonal,
    mode='lines',
    name='Seasonal',
    line=dict(color='orange')
))

# Add the residual component
fig.add_trace(go.Scatter(
    x=dates,
    y=residual,
    mode='lines',
    name='Residual',
    line=dict(color='green')
))

# Customize the layout
fig.update_layout(
    title='Time Series Decomposition of Sales Revenue',
    xaxis_title='Date',
    yaxis_title='Value',
    hovermode='x',  # Interactive hover on the x-axis
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig.show()

In [19]:
# Ensure 'quantity' is not empty and drop NaN values
quantity_data = daily_sales['quantity'].dropna()

# Decompose the time series (ensure you have enough data points)
if len(quantity_data) >= 24:
    decomposition = sm.tsa.seasonal_decompose(quantity_data, model='additive')

    # Extract the components
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    dates = quantity_data.index  # Use the index of the original series

    # Create the interactive plot
    fig = go.Figure()

    # Add the trend component
    fig.add_trace(go.Scatter(
        x=dates,
        y=trend,
        mode='lines',
        name='Trend',
        line=dict(color='blue')
    ))

    # Add the seasonal component
    fig.add_trace(go.Scatter(
        x=dates,
        y=seasonal,
        mode='lines',
        name='Seasonal',
        line=dict(color='orange')
    ))

    # Add the residual component
    fig.add_trace(go.Scatter(
        x=dates,
        y=residual,
        mode='lines',
        name='Residual',
        line=dict(color='green')
    ))

    # Customize the layout
    fig.update_layout(
        title='Time Series Decomposition of Sales Quantity',
        xaxis_title='Date',
        yaxis_title='Quantity Sold',
        hovermode='x',  # Interactive hover on the x-axis
        template='plotly_white',
        width=1000,
        height=600
    )

    # Show the interactive plot
    fig.show()

In [20]:
# Ensure the sales revenue data is available
sales_data = monthly_sales['order_total_products_price'].dropna()

# Check the length of the data
if len(sales_data) < 24:
    print("Not enough data for seasonal decomposition. Using rolling mean instead.")

# Calculate rolling mean for trend (using a window of 3 for a simple trend)
trend = sales_data.rolling(window=3).mean()

# Calculate residuals as the difference from the rolling mean
residual = sales_data - trend

# Create the interactive plot for trend and residual
fig = go.Figure()

# Add the actual sales data
fig.add_trace(go.Scatter(
    x=sales_data.index,
    y=sales_data,
    mode='lines+markers',
    name='Actual Sales',
    line=dict(color='gray')
))

# Add the trend component
fig.add_trace(go.Scatter(
    x=sales_data.index,
    y=trend,
    mode='lines',
    name='Trend',
    line=dict(color='blue')
))

# Add the residual component
fig.add_trace(go.Scatter(
    x=sales_data.index,
    y=residual,
    mode='lines',
    name='Residual',
    line=dict(color='green')
))

# Customize the layout
fig.update_layout(
    title='Sales Revenue Analysis',
    xaxis_title='Month',
    yaxis_title='Revenue',
    hovermode='x',
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig.show()

Not enough data for seasonal decomposition. Using rolling mean instead.


In [21]:
# Ensure the sales quantity data is available
quantity_data = monthly_sales['quantity'].dropna()

# Check the length of the data
if len(quantity_data) < 24:
    print("Not enough data for seasonal decomposition. Using rolling mean instead.")

# Calculate rolling mean for trend (using a window of 3 for a simple trend)
trend = quantity_data.rolling(window=3).mean()

# Calculate residuals as the difference from the rolling mean
residual = quantity_data - trend

# Create the interactive plot for trend and residual
fig = go.Figure()

# Add the actual sales quantity data
fig.add_trace(go.Scatter(
    x=quantity_data.index,
    y=quantity_data,
    mode='lines+markers',
    name='Actual Quantity Sold',
    line=dict(color='gray')
))

# Add the trend component
fig.add_trace(go.Scatter(
    x=quantity_data.index,
    y=trend,
    mode='lines',
    name='Trend',
    line=dict(color='blue')
))

# Add the residual component
fig.add_trace(go.Scatter(
    x=quantity_data.index,
    y=residual,
    mode='lines',
    name='Residual',
    line=dict(color='green')
))

# Customize the layout
fig.update_layout(
    title='Quantity Sold Analysis',
    xaxis_title='Month',
    yaxis_title='Quantity Sold',
    hovermode='x',
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig.show()

Not enough data for seasonal decomposition. Using rolling mean instead.


In [22]:
# Prepare data for Prophet
prophet_df = daily_sales[['order_total_products_price']].reset_index()
prophet_df.columns = ['ds', 'y']  # Rename columns as expected by Prophet

# Define the closed store dates
closed_days = pd.to_datetime([
    '2024-04-09', '2024-04-10', '2024-04-11', '2024-04-12', '2024-04-13',
    '2024-06-15', '2024-06-16', '2024-06-17', '2024-06-18', '2024-06-19', '2024-06-20', '2024-06-21',
    '2024-08-15',
    '2024-08-27', '2024-08-28', '2024-08-29', '2024-08-30'
])

# Filter out the closed days from prophet_df (removes closed days data)
prophet_df = prophet_df[~prophet_df['ds'].isin(closed_days)]

# Create a holidays DataFrame for the closed days
holidays = pd.DataFrame({
    'holiday': 'store_closed',
    'ds': closed_days,
    'lower_window': 0,
    'upper_window': 1
})

# Instantiate the model with holidays
model = Prophet(holidays=holidays)

# Fit the model with the sales data
model.fit(prophet_df)

# Create future dataframe for the next 90 days
future = model.make_future_dataframe(periods=90)

# Predict future sales
forecast = model.predict(future)

# Remove forecasted data for closed days
forecast_filtered = forecast[~forecast['ds'].isin(closed_days)]

# Create interactive plot
fig = go.Figure()

# Add historical data
fig.add_trace(go.Scatter(
    x=prophet_df['ds'],
    y=prophet_df['y'],
    mode='lines+markers',
    name='Historical Data',
    line=dict(color='blue')
))

# Add forecast data (filtered, without closed days)
fig.add_trace(go.Scatter(
    x=forecast_filtered['ds'],
    y=forecast_filtered['yhat'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange')
))

# Add uncertainty intervals (filtered, without closed days)
fig.add_trace(go.Scatter(
    x=forecast_filtered['ds'],
    y=forecast_filtered['yhat_lower'],
    mode='lines',
    name='Lower Confidence Interval',
    line=dict(color='gray', dash='dash')
))

fig.add_trace(go.Scatter(
    x=forecast_filtered['ds'],
    y=forecast_filtered['yhat_upper'],
    mode='lines',
    name='Upper Confidence Interval',
    line=dict(color='gray', dash='dash')
))

# Customize the layout
fig.update_layout(
    title='Sales Forecast',
    xaxis_title='Date',
    yaxis_title='Sales Revenue',
    hovermode='x',  # Interactive hover on the x-axis
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig.show()


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/j_f3fs36.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/vsrq0zuy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=17918', 'data', 'file=/tmp/tmpdemtqu8u/j_f3fs36.json', 'init=/tmp/tmpdemtqu8u/vsrq0zuy.json', 'output', 'file=/tmp/tmpdemtqu8u/prophet_modela7p3e_dt/prophet_model-20240926071524.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:15:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:15:24 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [29]:
# Save the Plotly figure as a JSON file
pio.write_json(fig, 'sales_forecast_plot (Organic).json')

print("Plot saved as sales_forecast_plot.json")

Plot saved as sales_forecast_plot.json


In [23]:
# Prepare data for Prophet
prophet_df2 = daily_sales[['quantity']].reset_index()
prophet_df2.columns = ['ds', 'y']  # Rename columns as expected by Prophet

# Instantiate and fit the model
model2 = Prophet()
model2.fit(prophet_df2)

# Create future dataframe (e.g., next 90 days)
future2 = model2.make_future_dataframe(periods=90)

# Predict future sales
forecast2 = model2.predict(future2)

# Create interactive plot
fig2 = go.Figure()

# Add historical data
fig2.add_trace(go.Scatter(
    x=prophet_df2['ds'],
    y=prophet_df2['y'],
    mode='lines+markers',
    name='Historical Data',
    line=dict(color='blue')
))

# Add forecast data
fig2.add_trace(go.Scatter(
    x=forecast2['ds'],
    y=forecast2['yhat'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange')
))

# Add lower confidence interval
fig2.add_trace(go.Scatter(
    x=forecast2['ds'],
    y=forecast2['yhat_lower'],
    mode='lines',
    name='Lower Confidence Interval',
    line=dict(color='gray', dash='dash')
))

# Add upper confidence interval
fig2.add_trace(go.Scatter(
    x=forecast2['ds'],
    y=forecast2['yhat_upper'],
    mode='lines',
    name='Upper Confidence Interval',
    line=dict(color='gray', dash='dash')
))

# Customize the layout
fig2.update_layout(
    title='Sales Quantity Forecast',
    xaxis_title='Date',
    yaxis_title='Quantity Sold',
    hovermode='x',  # Interactive hover on the x-axis
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig2.show()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/16tr6joq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/um_hm4kp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=22105', 'data', 'file=/tmp/tmpdemtqu8u/16tr6joq.json', 'init=/tmp/tmpdemtqu8u/um_hm4kp.json', 'output', 'file=/tmp/tmpdemtqu8u/prophet_modelt0m77bu_/prophet_model-20240926071525.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:15:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:15:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [24]:
# Step 1: Aggregate the data to count the number of orders per day
orders_per_day = df.groupby('order_date')['order_id'].nunique().reset_index()

# Step 2: Prepare data for Prophet
prophet_df_3 = orders_per_day.rename(columns={'order_date': 'ds', 'order_id': 'y'})  # Rename columns for Prophet

# Step 3: Instantiate and fit the Prophet model
model_3 = Prophet()
model_3.fit(prophet_df_3)

# Step 4: Create a future dataframe (e.g., next 90 days)
future_3 = model_3.make_future_dataframe(periods=90)

# Step 5: Predict future orders
forecast_3 = model_3.predict(future_3)

# Step 6: Create an interactive plot
fig_3 = go.Figure()

# Add historical order data
fig_3.add_trace(go.Scatter(
    x=prophet_df_3['ds'],
    y=prophet_df_3['y'],
    mode='lines+markers',
    name='Historical Orders',
    line=dict(color='blue')
))

# Add forecasted orders
fig_3.add_trace(go.Scatter(
    x=forecast_3['ds'],
    y=forecast_3['yhat'],
    mode='lines',
    name='Forecast',
    line=dict(color='orange')
))

# Add uncertainty intervals
fig_3.add_trace(go.Scatter(
    x=forecast_3['ds'],
    y=forecast_3['yhat_lower'],
    mode='lines',
    name='Lower Confidence Interval',
    line=dict(color='gray', dash='dash')
))

fig_3.add_trace(go.Scatter(
    x=forecast_3['ds'],
    y=forecast_3['yhat_upper'],
    mode='lines',
    name='Upper Confidence Interval',
    line=dict(color='gray', dash='dash')
))

# Customize the layout
fig_3.update_layout(
    title='Order Forecast (Number of Orders per Day)',
    xaxis_title='Date',
    yaxis_title='Number of Orders',
    hovermode='x',
    template='plotly_white',
    width=1000,
    height=600
)

# Show the interactive plot
fig_3.show()


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/rcn_j7w2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpdemtqu8u/t97ycash.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=51182', 'data', 'file=/tmp/tmpdemtqu8u/rcn_j7w2.json', 'init=/tmp/tmpdemtqu8u/t97ycash.json', 'output', 'file=/tmp/tmpdemtqu8u/prophet_modelxse3fe20/prophet_model-20240926071525.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:15:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:15:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [26]:
# handling season drops [checked]
# association rules for the dataset( Organic ) [on progress]
